In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret():

    secret_name = "DBCreds"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    # Parse the secret string to get the credentials
    secret_dict = json.loads(secret)
    username = secret_dict['username']
    password = secret_dict['password']
    host = secret_dict['host']
    port = secret_dict['port']
    dbname = secret_dict['dbname']

    return username, password, host, port, dbname


(user,pswd,host,port,db) = get_secret()

In [ ]:
import json
from kafka import KafkaConsumer
import psycopg2
from psycopg2.extras import execute_values

# PostgreSQL database connection details
db_config = {
    'dbname': db,
    'user': user,
    'password': pswd,
    'host': host,
    'port': port
}

# Kafka topic
postgres_topic = "cyd-postgresql"
brokers = [f"{host}:9092"]

# Create a Kafka consumer
consumer = KafkaConsumer(
    postgres_topic,
    bootstrap_servers=brokers,
    value_deserializer=lambda message: json.loads(message.decode('utf-8'))
)

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully.")
except Exception as e:
    print(f"Failed to connect to PostgreSQL: {e}")
    exit()

try:
    print("Starting to consume messages.")
    for message in consumer:
        data = message.value
        #print(f"Received message: {data['outcome']}")
        uid = data.pop('uid')
        print(f"Received {uid} message: {data['outcome']}")
        data.pop('outcome')
        try:
            insert_query = """
            INSERT INTO cyber_data (uid, features)
            VALUES (%s, %s)
            """
            cursor.execute(insert_query, (uid, json.dumps(data)))
            conn.commit()
            print("Data inserted successfully.")
        except Exception as e:
            print(f"Failed to insert data: {e}")
            conn.rollback()
        
        
except KeyboardInterrupt:
    print("Consumer interrupted.")
finally:
    # Close PostgreSQL connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
        print("PostgreSQL connection closed.")


Connected to PostgreSQL successfully.
Starting to consume messages.
Received 4fef4758-a848-4f2a-aa5d-d1b08705a6c8 message: 5.0
Data inserted successfully.
Received e8a5a465-aaaf-479d-b39f-cebe8e57a481 message: 6.0
Data inserted successfully.
Received a1167c75-32fa-4c9d-97ba-27b2f5275b49 message: 5.0
Data inserted successfully.
Received 00a88961-bf2e-4646-95bc-155bc923f9f5 message: 3.0
Data inserted successfully.
Received d655f8dc-190d-4dcb-9e63-b4b41e54e73d message: 3.0
Data inserted successfully.
Received db6ca09a-8be3-4829-af0d-859bbad59c1e message: 5.0
Data inserted successfully.
Received 555cca63-363e-443f-a27a-157420721a43 message: 2.0
Data inserted successfully.
Received d9d19b3c-2bae-4067-ac0d-e8c3939b7e28 message: 6.0
Data inserted successfully.
Received 79cbf6e3-63bc-44c8-aa04-412b5588a6a2 message: 6.0
Data inserted successfully.
Received 1971bd95-c55c-45f5-8f81-92b9dbd80088 message: 5.0
Data inserted successfully.
Received 4f7f5c96-eeeb-43ee-8f8f-1b12eaba35ab message: 6.0
Dat